# Weighted MaxEnt Species Distribution Model Training and Prediction

This notebook implements a **weighted version** of the MaxEnt (Maximum Entropy) species distribution modealing workflow. Unlike the standard MaxEnt approach, this version incorporates **sample weights** to account for data quality, spatial bias, or temporal differences in occurrence records.

## Key Features of Weighted MaxEnt:

### 1. **Sample Weighting**:
- **Data Quality Weights**: Weight samples based on data source reliability
- **Spatial Bias Correction**: Reduce influence of oversampled regions
- **Temporal Weights**: Account for temporal bias in occurrence records
- **Expert Knowledge Integration**: Incorporate expert assessments of record quality

### 2. **Weighting Strategies**:
- **Source-based**: Different weights for GBIF, CABI, research publications
- **Spatial**: Inverse distance weighting or kernel density weighting
- **Temporal**: Recent records weighted higher than historical ones
- **Quality-based**: Expert-assessed data quality scores

### 3. **Model Configuration**:
- **Algorithm**: Weighted MaxEnt with logistic output transformation
- **Regularization**: Beta multiplier = 1.5 (controls model complexity)
- **Weight Integration**: Weights applied during model training
- **Output**: Relative occurrence probability (0-1 scale)

## Applications:
- **Bias Correction**: Reduce spatial and temporal sampling bias
- **Data Integration**: Combine multiple data sources with different quality
- **Expert Knowledge**: Incorporate field expert assessments
- **Quality Control**: Emphasize high-quality occurrence records

In [ ]:
############### WEIGHTED MODEL CONFIGURATION - MODIFY AS NEEDED ###############

# Species and region settings for weighted modeling
#specie = 'leptocybe-invasa'  # Target species: 'leptocybe-invasa' or 'thaumastocoris-peregrinus'
#pseudoabsence = 'random'  # Background point strategy: 'random', 'biased', 'biased-land-cover'
#training = 'india-sri-lanka'  # Training region: 'sea', 'australia', 'east-asia', etc.
#interest = 'south-east-asia'  # Test region: can be same as training or different

# Environmental variable configuration
# bioclim = bioclim_model  # Bioclimatic variables (from previous notebook)
bioclim = bioclim  # Current bioclimatic variable set
bio = bio1  # Bioclimatic variable identifier

# Additional environmental variables
topo = topo1  # Topographic variables (elevation, slope, aspect)
ndvi = ndvi1  # Normalized Difference Vegetation Index
# topo = True  # Alternative: include all topographic variables

# Output settings
savefig = True  # Save generated maps as PNG files

###########################################################

In [ ]:
# =============================================================================
# IMPORT REQUIRED LIBRARIES
# =============================================================================

import os  # File system operations

import math  # Mathematical functions
import numpy as np  # Numerical computing

import xarray as xr  # Multi-dimensional labeled arrays (raster data)
import rioxarray as rioxr  # Raster I/O for xarray
import pandas as pd  # Data manipulation and analysis
import geopandas as gpd  # Geospatial data handling

import elapid as ela  # Species distribution modeling library
from sklearn import metrics, inspection  # Machine learning metrics and model inspection

import matplotlib.pyplot as plt  # Plotting and visualization
import matplotlib.colors as mcolors  # Color mapping utilities

import warnings
warnings.filterwarnings("ignore")  # Suppress warning messages for cleaner output

# Configure matplotlib for publication-quality plots
params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

In [ ]:
def subplot_layout(nplots):
    """
    Calculate optimal subplot layout for given number of plots
    
    Parameters:
    -----------
    nplots : int
        Number of plots to arrange
    
    Returns:
    --------
    ncols, nrows : tuple
        Number of columns and rows for subplot layout
    """
    
    # Calculate square root and round up for balanced layout
    ncols = min(int(np.ceil(np.sqrt(nplots))), 4)  # Max 4 columns
    nrows = int(np.ceil(nplots / ncols))  # Calculate rows needed
    
    return ncols, nrows

In [ ]:
# =============================================================================
# SET UP FILE PATHS
# =============================================================================
# Define directory structure for organizing weighted model outputs

# data_path = os.path.join(os.sep, 'scratch', 'aciar-fst', 'data')  # Alternative server path
data_path = os.path.join(os.path.dirname(os.getcwd()), 'data')  # Main data directory
figs_path = os.path.join(os.path.dirname(os.getcwd()), 'figs')  # Figures directory
docs_path = os.path.join(os.path.dirname(os.getcwd()), 'docs')  # Documentation directory
out_path = os.path.join(os.path.dirname(os.getcwd()), 'out', specie)  # Species-specific output directory
input_path = os.path.join(out_path, 'input')  # Input data directory
train_path = os.path.join(input_path, 'train')  # Training data directory
test_path = os.path.join(input_path, 'test')  # Test data directory
output_path = os.path.join(out_path, 'output')  # Model output directory

# Create output directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

## Load Geographic Boundaries

Load shapefiles defining the training and test regions for the weighted MaxEnt model. These boundaries are used to:
- **Spatial Cropping**: Extract environmental data for specific regions
- **Model Training**: Define the geographic extent for model training
- **Model Testing**: Apply trained models to different geographic areas
- **Visualization**: Provide geographic context for distribution maps

In [ ]:
# =============================================================================
# LOAD TRAINING REGION BOUNDARIES
# =============================================================================
# Load shapefile defining the training region for weighted model development

# Alternative approach: Load multiple regions (commented out)
# gdf_countries = {}
# for region in [training, interest]:
#     file_path = train_path if region == training else test_path
#     gdf_countries[region] = gpd.read_file(os.path.join(file_path, '%s.shp' %region))

# Load training region shapefile
gdf_countries = gpd.read_file(os.path.join(input_path, '%s.shp' %training))

## 1. Train Weighted MaxEnt Model

This section trains the weighted MaxEnt model using occurrence data from the training region. The key difference from standard MaxEnt is the incorporation of **sample weights** to account for:

- **Data Source Quality**: Different weights for GBIF, CABI, research publications
- **Spatial Bias**: Reduce influence of oversampled areas
- **Temporal Bias**: Weight recent records higher than historical ones
- **Expert Knowledge**: Incorporate field expert assessments of record quality

### Weighted MaxEnt Advantages:
- **Bias Reduction**: Mitigate spatial and temporal sampling bias
- **Data Integration**: Combine multiple data sources with different quality levels
- **Improved Accuracy**: Better model performance through quality-weighted training
- **Robust Predictions**: More reliable distribution estimates

### 1.1 load predictive variable data

In [ ]:
# Assemble predictor raster filenames and human-readable labels
# - Includes optional topography (SRTM) and NDVI depending on `topo` and `ndvi`
# - Appends selected bioclim variables with a `model_prefix` tag for traceability
# rasters, labels = (['srtm_%s.tif' %training], ['srtm']) if topo else ([], []) # 'ndvi_east-asia.tif', 'ndvi' # 

# Base layers (topography)
rasters, labels = (
    (['srtm_%s.tif' % training], ['srtm']) if topo else ([], [])
)
# Optional NDVI layer
rasters, labels = (
    rasters + (['ndvi_%s.tif' % training] if ndvi else []),
    labels  + (['ndvi'] if ndvi else [])
)

# Historical vs future template (kept for reference)
# if Future:
#     for no in bioclim:
#         rasters.append('bio_%s_%s_future.tif' %(no, training))
#         labels.append('bioclim_%02d' %no)
# else:
#     for no in bioclim:
#         rasters.append('bio_%s_%s.tif' %(no, training))
#         labels.append('bioclim_%02d' %no)

# Selected bioclim features (current)
for no in bioclim:
    rasters.append('%s_bio_%s_%s.tif' %(model_prefix, no, training))
    labels.append('%s_bioclim_%02d' %(model_prefix, no))

# Resolve full file paths for raster IO
raster_paths = [os.path.join(input_path, raster) for raster in rasters]
# rasters, raster_paths, labels

# Initialize xarray.Dataset to carry predictors and later predictions
training_output = xr.Dataset()
for raster, label in zip(raster_paths, labels):
    # Use masked read to preserve nodata as NaN
    da = rioxr.open_rasterio(raster, masked=True)
    training_output[label] = da

# print(label)
# print(da)

# # Example plotting block (disabled)
# num_plots = len(labels)
# fig, axes = plt.subplots(4, 5, figsize=(20, 16))  # 4 rows, 5 cols
# axes = axes.flatten()
# for ax, label in zip(axes, labels):
#     training_data[label].plot(ax=ax, cmap='viridis')
#     ax.set_title(label)
#     ax.axis('off')
# for ax in axes[len(labels):]:
#     ax.axis('off')
# plt.tight_layout(); plt.show()

# print(training_output)
# training_output.to_netcdf('../data/training_output.nc')

### 1.2 load and merge presence and background data

In [ ]:
# Build canonical input filenames for presence, background, and exported train table
# Presence file includes occurrence coordinates for the training region
presence_file_name = '%s_presence_%s_%s.csv' %(specie, training, iteration)
# Background file depends on pseudoabsence strategy label
background_file_name = '%s_background_%s_%s.csv' %(specie, pseudoabsence, training)
# Name for the fully-merged training table saved for reproducibility
train_input_data_name = '%s_model-train_input-data_%s_%s_%s_%s_%s.csv' %(model_prefix, specie, pseudoabsence, training, bio, iteration)

# Load presence/background CSVs and convert lon/lat into a GeoDataFrame in WGS84
# Expect input columns 'lon' and 'lat' in decimal degrees
presence_csv = pd.read_csv(os.path.join(input_path, 'train', presence_file_name))
geometry = gpd.points_from_xy(presence_csv['lon'], presence_csv['lat'])
presence_gdf = gpd.GeoDataFrame(geometry=geometry, crs='EPSG:4326')

# Background points used as pseudo-absences for MaxEnt
background_csv = pd.read_csv(os.path.join(input_path, 'train', background_file_name))
geometry = gpd.points_from_xy(background_csv['lon'], background_csv['lat'])
background_gdf = gpd.GeoDataFrame(geometry=geometry, crs='EPSG:4326')

In [ ]:
# Extract raster values at presence locations; drop observations that fall on nodata
# Returns a GeoDataFrame with columns for each predictor at each point
presence_train = ela.annotate(
    presence_gdf.geometry,
    raster_paths=raster_paths, 
    labels=labels, 
    drop_na=True,
)

# Compute sample weights for presence via distance-based weighting
# n_neighbors=-1 uses all other points to estimate local density (global KDE-like)
# Higher density => lower weight; isolated points => higher weight
presence_train['SampleWeight'] = ela.distance_weights(presence_train, n_neighbors=-1)
# Quick diagnostic plot of weights
presence_train.plot(column='SampleWeight', legend=True, cmap='YlOrBr')

# Extract raster values at background locations (pseudo-absences)
background_train = ela.annotate(
    background_gdf,  # supports different biasing strategies upstream
    raster_paths=raster_paths, 
    labels=labels, 
    drop_na=True,
)

# Background weights based on nearest-neighbor density (n_neighbors=1)
background_train['SampleWeight'] = ela.distance_weights(background_train, n_neighbors=1)

# Combine presence and background with a binary class label (1=presence, 0=background)
# Resulting table contains predictors, class label, geometry, and SampleWeight
train = ela.stack_geodataframes(
    presence_train,
    background_train,
    add_class_label=True,
)

In [ ]:
train.describe()

### 1.3 run model

In [ ]:
print("1.3 Run Model")
# Build experiment/run identifiers to keep outputs organized and reproducible
experiment_name = 'exp_%s_%s_%s_%s_%s' %(model_prefix, pseudoabsence, training, topo, ndvi)
run_name = '%s_model-train_%s_%s_%s_%s_%s.ela' %(model_prefix, specie, pseudoabsence, training, bio, iteration)
# NetCDF and raster file names for gridded predictions
nc_name = '%s_model-train_%s_%s_%s_%s_%s.nc' %(model_prefix, specie, pseudoabsence, training, bio, iteration)
raster_name = '%s_model-train_%s_%s_%s_%s_%s.tif' %(model_prefix, specie, pseudoabsence, training, bio, iteration)

# Make sure the experiment output directory exists
exp_path = os.path.join(output_path, experiment_name)
if not os.path.exists(exp_path):
    os.makedirs(exp_path)

In [ ]:
# Persist the fully-prepared training table for traceability/reproducibility
# Includes predictors, labels, geometry, and computed SampleWeight
train.to_csv(os.path.join(exp_path, train_input_data_name))

# Split features/labels/weights; drop non-feature columns
# x_train contains only predictor columns used by the model
x_train = train.drop(columns=['class', 'SampleWeight', 'geometry' ])
# y_train is the binary target: 1 for presence, 0 for background
y_train = train['class']
# sample_weight per observation passed to the learner to correct bias/imbalance
sample_weight_train = train['SampleWeight']

# Configure and fit weighted MaxEnt (logistic output)
# beta_multiplier controls regularization strength (higher => smoother model)
model_train = ela.MaxentModel(transform='logistic', beta_multiplier=1.5,)
print(model_train.get_params())
model_train.fit(x_train, y_train, sample_weight=sample_weight_train)

# Serialize trained model to disk for later reuse
ela.save_object(model_train, os.path.join(exp_path, run_name))

In [ ]:
# predict the model
print("predict the model")

# Point-wise predictions for the training table (used for metrics/diagnostics)
y_train_predict = model_train.predict(x_train)
# y_train_predict = np.nan_to_num(y_train_predict, nan=0.5)

# Apply the model across the full raster stack to produce a map
# Convert xarray Dataset to a 3D numpy array [band, y, x]
array = training_output.isel(band=0).to_array().values
# Define nodata value and mask for safe application over missing areas
nodata = np.nan
nodata_idx = np.isnan(array)
# Evaluate the model over the raster cube
rop = ela.geo.apply_model_to_array(model_train, array, nodata, nodata_idx)

# Attach the predicted probability map to the dataset under variable 'rop'
training_output['rop'] = (('band', 'y', 'x'), rop)
training_output['rop'].attrs['long_name'] = "relative occurrence probability"

# write model output to netcdf
training_output.to_netcdf(os.path.join(exp_path, nc_name))

# write model predictions to raster
ela.apply_model_to_rasters(model_train, raster_paths, os.path.join(exp_path, raster_name), quiet=True)

In [ ]:
# Model training performance metrics
# - ROC-AUC (unweighted and weighted by sample weights)
# - PR-AUC (unweighted and weighted)

# ROC-curve inputs (kept commented; AUC via direct score API below)
# fpr_train, tpr_train, thresholds = metrics.roc_curve(y_train, y_train_predict)

auc_train = metrics.roc_auc_score(y_train, y_train_predict)
auc_train_weighted = metrics.roc_auc_score(y_train, y_train_predict, sample_weight=sample_weight_train)

# PR-curve
precision_train, recall_train, _= metrics.precision_recall_curve(y_train, y_train_predict) 
pr_auc_train = metrics.auc(recall_train, precision_train)
precision_train_w, recall_train_w, _= metrics.precision_recall_curve(y_train, y_train_predict, sample_weight=sample_weight_train)
pr_auc_train_weighted = metrics.auc(recall_train, precision_train)

print(f"Training ROC-AUC score: {auc_train:0.3f}")
print(f"Training ROC-AUC Weighted score  : {auc_train_weighted:0.3f}")
print(f"PR-AUC Score: {pr_auc_train:0.3f}")
print(f"PR-AUC Weighted Score: {pr_auc_train_weighted:0.3f}")

In [ ]:
# print("Permutation_importace_Plot")
# model_train.permutation_importance_plot(x_train, y_train, sample_weight=sample_weight_train, n_repeats=10)

## 2. Predict model for testing data

### 2.1 load test data

In [ ]:
print("2.1 load test data")

# Rebuild predictor stack for the test/interest region
rasters, labels = (
    (['srtm_%s.tif' % training], ['srtm']) if topo else ([], [])
)
rasters, labels = (
    rasters + (['ndvi_%s.tif' % training] if ndvi else []),
    labels  + (['ndvi'] if ndvi else [])
)

# Templates for interest region or future runs are kept for reference above

# Add selected bioclim variables for the test prediction stack
for no in bioclim:
    rasters.append('%s_bio_%s_%s.tif' %(model_prefix, no, training))
    labels.append('%s_bioclim_%02d' %(model_prefix, no))

# Full paths to geotiffs to be opened
raster_paths = [os.path.join(input_path, raster) for raster in rasters]

# initialise dataset for model output
# Each opened raster becomes a variable in the dataset keyed by label
test_output = xr.Dataset()
for raster, label in zip(raster_paths, labels):
    da = rioxr.open_rasterio(raster, masked=True)
    test_output[label] = da

In [ ]:
# Build filenames for test presence/background and export name
presence_file_name = '%s_presence_%s_%s.csv' %(specie, interest, iteration)
background_file_name = '%s_background_%s_%s.csv' %(specie, pseudoabsence, interest)
# Export name for merged test table
test_input_data_name = '%s_model-test_input-data_%s_%s_%s_%s_%s.csv' %(model_prefix, specie, pseudoabsence, interest, bio, iteration)

#####----------------------------
# if Future:
#     presence_csv = pd.read_csv(os.path.join(input_path, 'test', presence_file_name))
#     geometry = gpd.points_from_xy(presence_csv['lon'], presence_csv['lat'])
#     presence_gdf = gpd.GeoDataFrame(geometry=geometry, crs='EPSG:4326')
#     print(presence_gdf.geometry)
#     print(presence_gdf)
#     presence_gdf = presence_gdf.drop(presence_gdf.index)
    
# else:
# Load point data for the interest region and convert to GeoDataFrames
presence_csv = pd.read_csv(os.path.join(input_path, 'test', presence_file_name))
geometry = gpd.points_from_xy(presence_csv['lon'], presence_csv['lat'])
presence_gdf = gpd.GeoDataFrame(geometry=geometry, crs='EPSG:4326')
        
background_csv = pd.read_csv(os.path.join(input_path, 'test', background_file_name))
geometry = gpd.points_from_xy(background_csv['lon'], background_csv['lat'])
background_gdf = gpd.GeoDataFrame(geometry=geometry, crs='EPSG:4326')

In [ ]:
# Handle edge-case: no presence points in interest region
# Downstream code expects at least one point, so insert a dummy coordinate
if len(presence_gdf) == 0:
    print('There are no occurrences of this specie in this region!')
    # chose arbitrary coordinate to make the code run (lon=115, lat=0)
    presence_gdf.geometry = gpd.points_from_xy([115], [0])

In [ ]:
# Annotate predictors at test presence points
presence_test = ela.annotate(
    presence_gdf.geometry,
    raster_paths=raster_paths, 
    labels=labels, 
    drop_na=True,
)

# Distance-based sample weights for presence (use all neighbors)
presence_test['SampleWeight'] = ela.distance_weights(presence_test, n_neighbors=-1)
# Quick visual check of presence weights
presence_test.plot(column='SampleWeight', legend=True, cmap='YlOrBr')

# Annotate predictors at test background points
background_test = ela.annotate(
    background_gdf,  # supports different biasing strategies upstream
    raster_paths=raster_paths, 
    labels=labels, 
    drop_na=True,
)

# Nearest-neighbor based weights for background
background_test['SampleWeight'] = ela.distance_weights(background_test, n_neighbors=1)

# Merge presence and background with binary class label
test = ela.stack_geodataframes(
    presence_test,
    background_test,
    add_class_label=True,
)

# Save merged test table for reproducibility
__ = test.to_csv(os.path.join(exp_path, test_input_data_name))

### 2.2 model predict for region of interest/ test data

In [ ]:
nc_name = '%s_model-test_%s_%s_%s_%s_%s.nc' %(model_prefix, specie, pseudoabsence, interest, bio, iteration)
raster_name = '%s_model-test_%s_%s_%s_%s_%s.tif' %(model_prefix, specie, pseudoabsence, interest, bio, iteration)

In [ ]:
print("2.2 model predict for region of interest/ test data")

# Split features/labels/weights for test set
# Remove non-feature columns and extract target + weights
x_test = test.drop(columns=['class', 'SampleWeight', 'geometry'])
y_test = test['class']
sample_weight_test = test['SampleWeight']

# Predict probabilities for test samples (tabular points)
y_test_predict = model_train.predict(x_test)
# y_test_predict = np.nan_to_num(y_test_predict, nan=0.5)

# Apply trained model across raster stack to produce a probability map
# Convert the predictor stack to 3D array [band, y, x]
array = test_output.isel(band=0).to_array().values
nodata = np.nan
nodata_idx = np.isnan(array)
rop = ela.geo.apply_model_to_array(model_train, array, nodata, nodata_idx)

# Store map into xarray and annotate
_test_da = (('band', 'y', 'x'), rop)
test_output['rop'] = _test_da
test_output['rop'].attrs['long_name'] = "relative occurrence probability"

# Persist outputs
# NetCDF: multi-variable dataset, GeoTIFF: single-band map
test_output.to_netcdf(os.path.join(exp_path, nc_name))
ela.apply_model_to_rasters(model_train, raster_paths, os.path.join(exp_path, raster_name), quiet=True)

### 2.3 test data performance

In [ ]:
print("2.3 test data performance")
# ROC-curve
# fpr_test, tpr_test, _ = metrics.roc_curve(y_test, y_test_predict)
auc_test = metrics.roc_auc_score(y_test, y_test_predict)
auc_test_weighted = metrics.roc_auc_score(y_test, y_test_predict, sample_weight=sample_weight_test)

# PR-curve
precision_test, recall_test, _= metrics.precision_recall_curve(y_test, y_test_predict) 
pr_auc_test = metrics.auc(recall_test, precision_test)
precision_test_w, recall_test_w, _= metrics.precision_recall_curve(y_test, y_test_predict, sample_weight=sample_weight_test)
pr_auc_test_weighted = metrics.auc(recall_test_w, precision_test_w)

print(f"Training ROC-AUC score: {auc_train:0.3f}")
print(f"Training ROC-AUC Weighted score: {auc_train_weighted:0.3f}")
print(f"Training PR-AUC Score: {pr_auc_train:0.3f}")
print(f"Training PR-AUC Weighted Score: {pr_auc_train_weighted:0.3f}")

print(f"Test ROC-AUC score: {auc_test:0.3f}")
print(f"Test ROC-AUC Weighted score: {auc_test_weighted:0.3f}")
print(f"Test PR-AUC Score: {pr_auc_test:0.3f}")
print(f"Test PR-AUC Weighted Score: {pr_auc_test_weighted:0.3f}")

In [ ]:
doc=False

fig, ax = plt.subplots(1, 2, figsize=(18, 8), constrained_layout=True) #dpi=100

cmap = plt.cm.GnBu #'GnBu'
bounds = np.linspace(0, 1, 11)
norm = mcolors.BoundaryNorm(bounds, cmap.N)

# pcol = train_out.plot(ax=ax[0], vmin=0, vmax=1, cmap='GnBu', add_colorbar=False)
gdf_countries.plot(ax=ax[0], facecolor='lightgray', edgecolor='k')
pcol = training_output.rop.plot(ax=ax[0], vmin=0, vmax=1, norm=norm, cmap=cmap, add_colorbar=False)
presence_train.plot(ax=ax[0], color='tab:red', marker='*', label='presence-train')
ax[0].text(0.98, 0.91, 'presence points: %s\n pseudo-absence points: %s\n AUC: %.3f' %(len(presence_train), len(background_train), auc_train_weighted), fontsize=12, 
           horizontalalignment='right', verticalalignment='top', transform=ax[0].transAxes)
ax[0].legend(loc='upper right')
ax[0].set_title('')
#ax[0].set_ylim([15, 60])

gdf_countries.plot(ax=ax[1], facecolor='lightgray', edgecolor='k')
pcol = test_output.rop.plot(ax=ax[1], vmin=0, vmax=1, norm=norm, cmap=cmap, add_colorbar=False)
presence_test.plot(ax=ax[1], color='tab:red', marker='*', label='presence-test')
ax[1].text(0.98, 0.92, 'presence points: %s\n pseudo-absence points: %s\n AUC: %.3f' %(len(presence_test), len(background_test), auc_test_weighted), fontsize=12, 
           horizontalalignment='right', verticalalignment='top', transform=ax[1].transAxes)
ax[1].legend()
ax[1].set_title('')

cbar = fig.colorbar(pcol, ax=ax, aspect=50, pad=0.05, label="relative occurrence probability", orientation='horizontal', fraction=0.03)

if doc:
    ax[0].axis('off')
    ax[1].axis('off')

if savefig:
    fig.savefig(os.path.join(docs_path, '05_rel-occ-prob-%s_%s_%s_%s_%s.png' %(specie, training, bio, model_prefix, iteration)), transparent=True, bbox_inches='tight')
        

In [ ]:
if savefig:
    # If a model is specified, add it to the filename
    file_path = os.path.join(figs_path, '05_rel-occ-prob-%s_%s_%s_%s_%s.png' %(specie, training, bio, model_prefix, iteration))
    fig.savefig(file_path, transparent=True, bbox_inches='tight')

### 2.4 load Future data

In [ ]:
print("2.4 Future Data")

rasters, labels = (
    (['srtm_%s.tif' % training], ['srtm']) if topo else ([], [])
)
rasters, labels = (
    rasters + (['ndvi_%s.tif' % training] if ndvi else []),
    labels  + (['ndvi'] if ndvi else [])
)
# rasters, labels = (['srtm_%s.tif' %interest], ['srtm']) if topo else ([], []) # 'ndvi_east-asia.tif', 'ndvi' # 

for no in bioclim:
    rasters.append('%s_bio_%s_%s_future.tif' %(model_prefix, no, training))
    labels.append('%s_bioclim_%02d' %(model_prefix, no))
    
raster_paths = [os.path.join(input_path, raster) for raster in rasters]

# initialise dataset for model output
# future_output = xr.Dataset()
# for raster, label in zip(raster_paths, labels):
#     da = rioxr.open_rasterio(raster, masked=True).squeeze(drop=True)  
#     future_output[label] = da


arrays = []
for raster in raster_paths:
    da = rioxr.open_rasterio(raster, masked=True)
    arr = da.squeeze().values  # pastikan 2D
    arrays.append(arr)


nc_name = '%s_model-future_%s_%s_%s_%s_%s.nc' %(model_prefix, specie, pseudoabsence, interest, bio, iteration)
raster_name = '%s_model-future_%s_%s_%s_%s_%s.tif' %(model_prefix, specie, pseudoabsence, interest, bio, iteration)

In [ ]:
stacked = np.stack(arrays, axis=-1)  # shape: (rows, cols, features)
nrow, ncol, nfeat = stacked.shape

X_future = stacked.reshape(-1, nfeat)

y_pred = model_train.predict(X_future)  # probabilitas presence
# y_pred = np.nan_to_num(y_pred, nan=0.5)

# Reshape raster 2D
y_map = y_pred.reshape(nrow, ncol)

future_output = xr.Dataset()
for raster, label in zip(raster_paths, labels):
    da = rioxr.open_rasterio(raster, masked=True)
    future_output[label] = da

future_output['rop'] = (('band', 'y', 'x'), y_map[np.newaxis, :, :])
future_output['rop'].attrs['long_name'] = "relative occurrence probability"

# --- 7. Simpan ke NetCDF ---
future_output.to_netcdf(os.path.join(exp_path, nc_name))

In [ ]:
doc=False

fig, ax = plt.subplots(1, 3, figsize=(24, 8), constrained_layout=True) #dpi=100

cmap = plt.cm.GnBu #'GnBu'
bounds = np.linspace(0, 1, 11)
norm = mcolors.BoundaryNorm(bounds, cmap.N)

# pcol = train_out.plot(ax=ax[0], vmin=0, vmax=1, cmap='GnBu', add_colorbar=False)
gdf_countries.plot(ax=ax[0], facecolor='lightgray', edgecolor='k')
pcol = training_output.rop.plot(ax=ax[0], vmin=0, vmax=1, norm=norm, cmap=cmap, add_colorbar=False)
presence_train.plot(ax=ax[0], color='tab:red', marker='*', label='presence-train')
ax[0].text(0.98, 0.91, 'presence points: %s\n pseudo-absence points: %s\n AUC: %.3f' %(len(presence_train), len(background_train), auc_train_weighted), fontsize=12, 
           horizontalalignment='right', verticalalignment='top', transform=ax[0].transAxes)
ax[0].legend(loc='upper right')
ax[0].set_title('')
#ax[0].set_ylim([15, 60])

gdf_countries.plot(ax=ax[1], facecolor='lightgray', edgecolor='k')
pcol = test_output.rop.plot(ax=ax[1], vmin=0, vmax=1, norm=norm, cmap=cmap, add_colorbar=False)
presence_test.plot(ax=ax[1], color='tab:red', marker='*', label='presence-test')
ax[1].text(0.98, 0.92, 'presence points: %s\n pseudo-absence points: %s\n AUC: %.3f' %(len(presence_test), len(background_test), auc_test_weighted), fontsize=12, 
           horizontalalignment='right', verticalalignment='top', transform=ax[1].transAxes)
ax[1].legend()
ax[1].set_title('')

gdf_countries.plot(ax=ax[2], facecolor='lightgray', edgecolor='k')
pcol = future_output.rop.plot(ax=ax[2], vmin=0, vmax=1, norm=norm, cmap=cmap, add_colorbar=False)
ax[2].set_title('Future prediction')

cbar = fig.colorbar(pcol, ax=ax, aspect=50, pad=0.05, label="relative occurrence probability", orientation='horizontal', fraction=0.03)

if doc:
    for a in ax:
        a.axis('off')

if savefig:
    fig.savefig(os.path.join(docs_path, '05_rel-occ-prob-%s_%s_%s_%s_%s_future.png' %(specie, training, bio, model_prefix, iteration)), transparent=True, bbox_inches='tight')
        

In [ ]:
if savefig:
    # If a model is specified, add it to the filename
    file_path = os.path.join(figs_path, '05_rel-occ-prob-%s_%s_%s_%s_%s_future.png' %(specie, training, bio, model_prefix, iteration))
    fig.savefig(file_path, transparent=True, bbox_inches='tight')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import xarray as xr

# --- ASSUMPTIONS: These variables already exist from previous code ---
# training_output.rop: xarray DataArray for training results
# future_output.rop: xarray DataArray for future/test results
# gdf_countries: GeoDataFrame for the background map
# -----------------------------------------------------------

## 1. Calculate the Difference
# We cannot directly subtract two xarray objects if their grids/coordinates differ.
# Use .reindex_like() to align 'future_output' to 'training_output'.
# Areas outside the coverage of 'future_output' will become NaN.
# future_aligned = future_output.rop.reindex_like(training_output.rop)

# Now we can safely subtract them
# Formula: Future Prediction - Historical/Training Prediction
difference = future_output.rop - training_output.rop

## 2. Prepare Visualization
fig, ax = plt.subplots(1, 1, figsize=(12, 8), constrained_layout=True)

# Choose a diverging colormap (centered around zero).
# 'coolwarm' (Blue-White-Red) or 'RdBu_r' works well.
# Blue = negative values, Red = positive values, White = zero.
cmap_diff = plt.cm.coolwarm

# Create a normalization centered at zero.
# This ensures that value 0 appears as white (neutral).
norm_diff = mcolors.CenteredNorm()

## 3. Plot the Difference
# Plot all countries as background
gdf_countries.plot(ax=ax, facecolor='lightgray', edgecolor='k')

# Plot raster difference on top
# xarray will automatically ignore NaN values
diff_plot = difference.plot(ax=ax, cmap=cmap_diff, norm=norm_diff, vmin=-1, vmax=1, add_colorbar=False)

# Add colorbar
cbar = fig.colorbar(diff_plot, ax=ax, orientation='vertical', label="Relative Occurrence Probability (Future - Historical)")

# Set title
ax.set_title('Change in Potentially Suitable Areas', fontsize=16)

plt.show()

In [ ]:
if savefig:
    # If a model is specified, add it to the filename
    file_path = os.path.join(figs_path, '05_rel-occ-dif-%s_%s_%s_%s_%s.png' %(specie, training, bio, model_prefix, iteration))
    fig.savefig(file_path, transparent=True, bbox_inches='tight')

In [ ]:
test_output.close()
future_output.close()